File Structure as follows:

*   content
  *   test_images
      *  1
          *  {48 photos from 5025 dataset}



In [ ]:
pip install facenet-pytorch

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
workers = 0 if os.name == 'nt' else 4

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [ ]:
mtcnn = MTCNN(
    image_size=300, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [ ]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
#resnet = InceptionResnetV1(pretrained='casia-webface').eval().to(device)
#resnet = InceptionResnetV1(num_classes=100).eval().to(device)

In [ ]:
def collate_fn(x):
  return x[0]

dataset = datasets.ImageFolder('/content/test_images/')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [ ]:
mtcnn_annotated = {}
for i,x in enumerate(loader):
  boxes, probs, landmarks = mtcnn.detect(x[0], landmarks=True)
  t = boxes
  l = landmarks
  boxes=boxes[0]
  landmarks=landmarks[0].tolist()
  list(np.float_(landmarks))

  bb = {}
  lm = {}
  bb = {"x":float(boxes[0]),"y":float(boxes[1]),"w":float(boxes[2])-float(boxes[0]),"h":float(boxes[3])-(boxes[1])}
  
  lm = {"left_eye": landmarks[0], "right_eye": landmarks[1], "nose": landmarks[2], "left_mouth": landmarks[3], "right_mouth": landmarks[4]}
  mtcnn_annotated[i] = {"bounding_box":bb,"landmarks":lm}
  


  
  fig, ax = plt.subplots(figsize=(16, 12))
  ax.imshow(x[0])
  ax.axis('off')
  boxes = t
  landmarks = l
  for box, landmark in zip(boxes, landmarks):

    ax.plot(box[[0, 2]],box[[1, 1]],'r-')
    ax.plot(box[[0, 2]],box[[3, 3]],'g-')
    ax.plot(box[[2, 2]],box[[1, 3]],'g-')
    ax.plot(box[[0, 0]],box[[3, 1]],'g-')



In [ ]:
print(mtcnn_annotated)

{0: {'bounding_box': {'x': 75.08161926269531, 'y': 49.37025451660156, 'w': 152.14666748046875, 'h': 197.97425842285156}, 'landmarks': {'left_eye': [118.37440490722656, 126.61767578125], 'right_eye': [187.8458251953125, 123.91761779785156], 'nose': [153.57379150390625, 159.09237670898438], 'left_mouth': [121.00163269042969, 195.56764221191406], 'right_mouth': [190.81802368164062, 193.5807647705078]}}, 1: {'bounding_box': {'x': 84.6247787475586, 'y': 67.37847137451172, 'w': 128.14293670654297, 'h': 169.55481719970703}, 'landmarks': {'left_eye': [120.29412841796875, 127.47575378417969], 'right_eye': [180.3223876953125, 130.4014129638672], 'nose': [150.1439208984375, 155.69985961914062], 'left_mouth': [116.19044494628906, 185.6077880859375], 'right_mouth': [178.720458984375, 190.32379150390625]}}, 2: {'bounding_box': {'x': 47.69538879394531, 'y': 73.45742797851562, 'w': 168.41163635253906, 'h': 227.72702026367188}, 'landmarks': {'left_eye': [90.57249450683594, 162.015625], 'right_eye': [17

In [ ]:
import json
json = json.dumps(mtcnn_annotated)
f = open("mtcnn_annotated.json","w")
f.write(json)
f.close()